# Flat Representations Tutorial

In [1]:
import sys

sys.path.append("../")

In [2]:
# Imports
import os
from pathlib import Path
import polars as pl, numpy as np
import polars.selectors as cs

from EventStream.data.dataset_polars import Dataset
from EventStream.evaluation.FT_task_baseline import load_flat_rep, fit_baseline_task_model

from sklearn.ensemble import RandomForestClassifier

In [3]:
ESD = Dataset.load(Path(os.getcwd()) / "processed/sample")

In [4]:
ESD.cache_flat_representation(
    subjects_per_output_file=None,
    feature_inclusion_frequency=None,
    do_overwrite=False,
    do_update=True,
)
flat_reps = load_flat_rep(ESD, window_sizes=['7d', '30d', 'FULL'])
display(flat_reps['train'].head().collect())

Flattening Splits:   0%|          | 0/3 [00:00<?, ?it/s]

Subject chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Subject chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Subject chunks:   0%|          | 0/1 [00:00<?, ?it/s]

subject_id,timestamp,static/eye_color/BLUE,static/eye_color/BROWN,static/eye_color/GREEN,static/eye_color/HAZEL,static/eye_color/UNK,7d/HR/HR/count,7d/HR/HR/has_values_count,7d/age/age/count,7d/age/age/has_values_count,7d/department/CARDIAC/count,7d/department/ORTHOPEDIC/count,7d/department/PULMONARY/count,7d/department/UNK/count,7d/lab_name/GCS__EQ_1/count,7d/lab_name/GCS__EQ_1/has_values_count,7d/lab_name/GCS__EQ_10/count,7d/lab_name/GCS__EQ_10/has_values_count,7d/lab_name/GCS__EQ_11/count,7d/lab_name/GCS__EQ_11/has_values_count,7d/lab_name/GCS__EQ_12/count,7d/lab_name/GCS__EQ_12/has_values_count,7d/lab_name/GCS__EQ_13/count,7d/lab_name/GCS__EQ_13/has_values_count,7d/lab_name/GCS__EQ_14/count,7d/lab_name/GCS__EQ_14/has_values_count,7d/lab_name/GCS__EQ_15/count,7d/lab_name/GCS__EQ_15/has_values_count,7d/lab_name/GCS__EQ_2/count,7d/lab_name/GCS__EQ_2/has_values_count,7d/lab_name/GCS__EQ_3/count,7d/lab_name/GCS__EQ_3/has_values_count,7d/lab_name/GCS__EQ_4/count,7d/lab_name/GCS__EQ_4/has_values_count,7d/lab_name/GCS__EQ_5/count,7d/lab_name/GCS__EQ_5/has_values_count,…,FULL/lab_name/GCS__EQ_8/min,FULL/lab_name/GCS__EQ_9/min,FULL/lab_name/SOFA__EQ_1/min,FULL/lab_name/SOFA__EQ_2/min,FULL/lab_name/SOFA__EQ_3/min,FULL/lab_name/SOFA__EQ_4/min,FULL/lab_name/SpO2/min,FULL/lab_name/UNK/min,FULL/lab_name/creatinine/min,FULL/lab_name/potassium/min,FULL/temp/temp/min,FULL/HR/HR/max,FULL/age/age/max,FULL/lab_name/GCS__EQ_1/max,FULL/lab_name/GCS__EQ_10/max,FULL/lab_name/GCS__EQ_11/max,FULL/lab_name/GCS__EQ_12/max,FULL/lab_name/GCS__EQ_13/max,FULL/lab_name/GCS__EQ_14/max,FULL/lab_name/GCS__EQ_15/max,FULL/lab_name/GCS__EQ_2/max,FULL/lab_name/GCS__EQ_3/max,FULL/lab_name/GCS__EQ_4/max,FULL/lab_name/GCS__EQ_5/max,FULL/lab_name/GCS__EQ_6/max,FULL/lab_name/GCS__EQ_7/max,FULL/lab_name/GCS__EQ_8/max,FULL/lab_name/GCS__EQ_9/max,FULL/lab_name/SOFA__EQ_1/max,FULL/lab_name/SOFA__EQ_2/max,FULL/lab_name/SOFA__EQ_3/max,FULL/lab_name/SOFA__EQ_4/max,FULL/lab_name/SpO2/max,FULL/lab_name/UNK/max,FULL/lab_name/creatinine/max,FULL/lab_name/potassium/max,FULL/temp/temp/max
u8,datetime[μs],bool,bool,bool,bool,bool,i64,i64,u32,u32,i64,i64,i64,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,2010-06-24 13:23:00,null,null,true,null,null,4,4,1,1,0,0,1,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,null,null,null,null,null,null,14.237596,null,null,null,0.926212,-0.840872,-0.558276,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,16.671087,null,null,null,1.246791
0,2010-06-24 14:23:00,null,null,true,null,null,5,5,2,2,0,0,1,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,null,null,null,null,null,null,14.237596,null,null,null,0.819353,-0.840872,-0.55825,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,16.671087,null,null,null,1.246791
0,2010-06-24 15:23:00,null,null,true,null,null,7,7,3,3,0,0,1,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,null,null,null,null,null,null,13.020849,null,null,null,0.552203,-0.840872,-0.558224,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,16.671087,null,null,null,1.246791
0,2010-06-24 16:23:00,null,null,true,null,null,9,9,4,4,0,0,1,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,null,null,null,null,null,null,13.020849,null,null,null,0.552203,-0.840872,-0.558199,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,16.671087,null,null,null,1.246791
0,2010-06-24 17:23:00,null,null,true,null,null,12,12,5,5,0,0,1,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,null,null,null,null,null,null,13.020849,null,null,null,0.552203,-0.840872,-0.558173,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,16.671087,null,null,null,1

In [5]:
task_df = (
    ESD.events_df
    .groupby('subject_id')
    .agg(pl.col('timestamp').sample().first().alias('end_time'))
    .with_columns(
        pl.lit(np.random.choice([0, 1, 2], size=len(ESD.subject_ids))).alias('label'),
        pl.lit(None, dtype=pl.Datetime).alias('start_time')
    )
).lazy()

Loading events from /home/mmd/Projects/EventStreamGPT/sample_data/processed/sample/events_df.parquet...


In [6]:
out = fit_baseline_task_model(
    task_df, 'label', ESD, n_samples=3, 
    model_cls = RandomForestClassifier,
    model_param_distributions={'min_samples_split': [2, 10, 100]},
    verbose=20,
    hyperparameter_search_budget=2,
    error_score='raise',
    window_size_options=['7d', '30d', 'FULL'],
    seed=1,
)

Checking for validity of window size options...
Loading representations for 7d, 30d, FULL
Loading datasets for train, tuning
Done with datasets for train, tuning with X of shape (90, 485) (elapsed: 0:00:01.360938)
Loading datasets for held_out
Done with datasets for held_out with X of shape (10, 485) (elapsed: 0:00:00.109677)
Fitting model!
Fitting 3 folds for each of 2 candidates, totalling 6 fits


/home/mmd/mambaforge/envs/ESGPT_cpu_test_2/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/mmd/mambaforge/envs/ESGPT_cpu_test_2/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:474: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/home/mmd/mambaforge/envs/ESGPT_cpu_test_2/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/mmd/mambaforge/envs/ESGPT_cpu_test_2/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:474: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/home/mmd/mambaforge/envs/ESGPT_cpu_test_2/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/mmd/mambaforge/envs/ESGPT_cpu_test_2/lib/python3.10/site-packages/sklearn/

RandomizedSearchCV(cv=3, error_score='raise',
                   estimator=Pipeline(steps=[('loading_features',
                                              <EventStream.evaluation.FT_task_baseline.ESDFlatFeatureLoader object at 0x7f994076d300>),
                                             ('scaling', 'passthrough'),
                                             ('imputation', 'passthrough'),
                                             ('reduce_dim', 'passthrough'),
                                             ('model',
                                              RandomForestClassifier())]),
                   n_iter=2, n_jobs=-1,
                   param_distributions=[{'imputation'...
                                         'loading_features__window_sizes': <EventStream.evaluation.FT_task_baseline.WindowSizeDist object at 0x7f994076ed70>,
                                         'model__min_samples_split': [2, 10,
                                                                      100],
                                         'reduce_dim': [SelectKBest(score_func=<function mutual_info_classif at 0x7f9940aa2cb0>)],
                                         'reduce_dim__k': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9940ae0220>,
                                         'scaling': [StandardScaler()]}],
                   random_state=1, verbose=20)

In [9]:
out.best_params_

{'imputation': SimpleImputer(fill_value=0, strategy='constant'),
 'imputation__add_indicator': False,
 'imputation__fill_value': 0,
 'imputation__strategy': 'constant',
 'loading_features__convert_to_mean_var': 0,
 'loading_features__feature_inclusion_frequency': 1.4297724879798385e-05,
 'loading_features__window_sizes': ['FULL', '7d', '30d'],
 'model__min_samples_split': 2,
 'reduce_dim': PCA(n_components=30),
 'reduce_dim__n_components': 30,
 'scaling': StandardScaler()}

In [10]:
out.best_estimator_

Pipeline(steps=[('loading_features',
                 <EventStream.evaluation.FT_task_baseline.ESDFlatFeatureLoader object at 0x7f993fcb74c0>),
                ('scaling', StandardScaler()),
                ('imputation',
                 SimpleImputer(fill_value=0, strategy='constant')),
                ('reduce_dim', PCA(n_components=30)),
                ('model', RandomForestClassifier())])